In [4]:
def point_in_polygon(point, polygon):
    """
    Determine if a point is inside a polygon using the Ray Casting method.
    
    Parameters:
    - point (list): [longitude, latitude]
    - polygon (list of list): [[longitude, latitude], ...]

    Returns:
    - bool: True if point is inside the polygon, otherwise False
    """
    
    x, y = point
    odd_nodes = False
    j = len(polygon) - 1  # The last vertex is the previous one to the first

    for i in range(len(polygon)):
        xi, yi = polygon[i]
        xj, yj = polygon[j]
        if yi < y and yj >= y or yj < y and yi >= y: 
            if xi + (y - yi) / (yj - yi) * (xj - xi) < x:
                odd_nodes = not odd_nodes
        j = i

    return odd_nodes


False

In [45]:
import os
import json
import copy

def align_geojson_to_reference(data, reference):
    """Align the QGIS GeoJSON data to match the structure of the reference."""
    if isinstance(reference, dict):
        # If the key exists in the reference but not in the data, add it with NULL value
        for key, value in reference.items():
            if key not in data:
                data[key] = None if not isinstance(value, (dict, list)) else copy.deepcopy(value)
            else:
                data[key] = align_geojson_to_reference(data[key], value)

        # If the key exists in the data but not in the reference, remove it
        extra_keys = set(data.keys()) - set(reference.keys())
        for key in extra_keys:
            del data[key]
    elif isinstance(reference, list) and len(reference) > 0:
        # Handle the features list as a special case
        # Apply the reference structure to each feature in the data
        ref_item = reference[0]
        for idx, item in enumerate(data):
            data[idx] = align_geojson_to_reference(item, ref_item)

    return data

def preprocess_geojsons(qgis_file_path, reference_file_path):
    """Preprocess the QGIS GeoJSON based on the reference sample."""
    with open(qgis_file_path, 'r') as qgis_file:
        qgis_data = json.load(qgis_file)
    with open(reference_file_path, 'r') as ref_file:
        reference_data = json.load(ref_file)
    
    aligned_data = align_geojson_to_reference(qgis_data, reference_data)
    return aligned_data

def set_feature_type(data, feature_type_name):
    """Set the feature_type for each feature in the GeoJSON data."""
    for feature in data.get("features", []):
        feature["feature_type"] = feature_type_name
        feature["type"] = "Feature"


    return data


def process_all_geojsons(input_dir, output_dir, reference_dir):
    """Process all GeoJSONs in the input directory to align them to the reference format."""
    for file_name in os.listdir(reference_dir):
        if file_name.endswith(".geojson"):
            qgis_file_path = os.path.join(input_dir, file_name)
            reference_file_path = os.path.join(reference_dir, file_name)
            
            if os.path.exists(qgis_file_path):  # If QGIS GeoJSON exists
                aligned_data = preprocess_geojsons(qgis_file_path, reference_file_path)
            else:  # If QGIS GeoJSON does not exist, just copy the reference
                with open(reference_file_path, 'r') as ref_file:
                    aligned_data = json.load(ref_file)
            
            # Set the feature_type for each feature to be the same as the name key
            feature_type_name = aligned_data.get("name", "")
            set_feature_type(aligned_data, feature_type_name)

            # Save the aligned data
            output_file_path = os.path.join(output_dir, file_name)
            with open(output_file_path, 'w') as output_file:
                json.dump(aligned_data, output_file, indent=4)
            print(f"Processed {qgis_file_path} -> {output_file_path}")


if __name__ == "__main__":
    input_dir = "C:\\Users\\amosk\\GitHub\\Geojson_2_IMDF_Transformer\\QGIS_Geojsons"
    output_dir = "C:\\Users\\amosk\\GitHub\\Geojson_2_IMDF_Transformer\\Transformed_QGIS_Geojsons"
    reference_dir = "C:\\Users\\amosk\\GitHub\\Geojson_2_IMDF_Transformer\\IMDF_Geojson_Cleaned_Sample"
    process_all_geojsons(input_dir, output_dir, reference_dir)


Processed C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\QGIS_Geojsons\address.geojson -> C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\address.geojson
Processed C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\QGIS_Geojsons\amenity.geojson -> C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\amenity.geojson
Processed C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\QGIS_Geojsons\anchor.geojson -> C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\anchor.geojson
Processed C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\QGIS_Geojsons\building.geojson -> C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\building.geojson
Processed C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\QGIS_Geojsons\footprint.geojson -> C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\footprint.geojson
Processed C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\QGIS_Geojsons\level.ge

In [50]:
import os
import json
import uuid

def convert_to_polygon(data):
    """Convert MultiPolygon geometries to Polygon."""
    for feature in data["features"]:
        if feature["geometry"]["type"] == "MultiPolygon":
            feature["geometry"]["type"] = "Polygon"
            feature["geometry"]["coordinates"] = feature["geometry"]["coordinates"][0]

def convert_to_linestring(data):
    """Convert MultiLineString geometries to LineString."""
    for feature in data["features"]:
        if feature["geometry"]["type"] == "MultiLineString":
            feature["geometry"]["type"] = "LineString"
            ## takes the first polygon of a multipolygon, which may contain holes. but usually is just one polygon
            feature["geometry"]["coordinates"] = feature["geometry"]["coordinates"][0] 

def point_in_polygon(point, polygon):
    """Check if a point is inside a polygon using the ray-casting algorithm."""
    x, y = point
    odd_nodes = False
    j = len(polygon) - 1  # The last vertex is the previous one to the first

    for i in range(len(polygon)):
        xi, yi = polygon[i]
        xj, yj = polygon[j]
        if yi < y and yj >= y or yj < y and yi >= y:
            if xi + (y - yi) / (yj - yi) * (xj - xi) < x:
                odd_nodes = not odd_nodes
        j = i
    return odd_nodes

def map_anchors_to_units(anchors, units):
    """Map anchors to units based on their spatial location."""
    for anchor in anchors["features"]:
        anchor_point = anchor["geometry"]["coordinates"]
        for unit in units["features"]:
            unit_polygon = unit["geometry"]["coordinates"][0]
            if point_in_polygon(anchor_point, unit_polygon):
                anchor["properties"]["unit_id"] = unit["id"]
                break  # Break out once a matching unit is found

def map_amenity_to_units(amenities, units):
    """Map amenities to units based on their spatial location."""
    for amenity in amenities["features"]:
        amenity_point = amenity["geometry"]["coordinates"]
        for unit in units["features"]:
            unit_polygon = unit["geometry"]["coordinates"][0]
            if point_in_polygon(amenity_point, unit_polygon):
                amenity["properties"]["unit_id"] = unit["id"]
                break  # Break out once a matching unit is found

def direct_id_mapping(features, target_features, mapping_rules):
    """Map the IDs directly based on the provided mapping rules."""
    target_ids = {feature["properties"]["name"]: feature["id"] for feature in target_features["features"]}
    for feature in features["features"]:
        for src_key, target_key in mapping_rules.items():
            if feature["properties"]["name"] in target_ids:
                feature["properties"][src_key] = target_ids[feature["properties"]["name"]]

def post_process_geojsons(directory):
    """Post-process all GeoJSONs in the directory."""
    units_data = None  # This will hold the unit data for mapping anchors
    
    # Process unit.geojson first
    unit_file_path = os.path.join(directory, "unit.geojson")
    if os.path.exists(unit_file_path):
        with open(unit_file_path, 'r') as file:
            units_data = json.load(file)
        convert_to_polygon(units_data)
        for feature in units_data["features"]:
            feature["id"] = str(uuid.uuid4())
        with open(unit_file_path, 'w') as output_file:
            json.dump(units_data, output_file, indent=4)
        print(f"Processed and overwrote {unit_file_path}")

    # Now process the anchor.geojson
    anchor_file_path = os.path.join(directory, "anchor.geojson")
    if os.path.exists(anchor_file_path) and units_data:
        with open(anchor_file_path, 'r') as file:
            anchors_data = json.load(file)
        for feature in anchors_data["features"]:
            feature["id"] = str(uuid.uuid4())
        map_anchors_to_units(anchors_data, units_data)
        with open(anchor_file_path, 'w') as output_file:
            json.dump(anchors_data, output_file, indent=4)
        print(f"Processed and overwrote {anchor_file_path}")

    with open(os.path.join(directory, "unit.geojson"), 'r') as file:
        units_data = json.load(file)
    with open(os.path.join(directory, "address.geojson"), 'r') as file:
        address_data = json.load(file)
    with open(os.path.join(directory, "level.geojson"), 'r') as file:
        level_data = json.load(file)
    with open(os.path.join(directory, "opening.geojson"), 'r') as file:
        openings_data = json.load(file)
    with open(os.path.join(directory, "venue.geojson"), 'r') as file:
        venues_data = json.load(file)

    
    # Map amenity to units
    amenity_file_path = os.path.join(directory, "amenity.geojson")
    with open(amenity_file_path, 'r') as file:
        amenities_data = json.load(file)
    map_amenity_to_units(amenities_data, units_data)
    with open(amenity_file_path, 'w') as file:
        json.dump(amenities_data, file)

    # Direct ID mappings
    direct_id_mapping(amenities_data, address_data, {"address_id": "id"})
    direct_id_mapping(anchors_data, address_data, {"address_id": "id"})
    direct_id_mapping(level_data, address_data, {"address_id": "id", "building_id": "id"})
    direct_id_mapping(openings_data, level_data, {"level_id": "id"})
    direct_id_mapping(units_data, level_data, {"level_id": "id"})
    direct_id_mapping(venues_data, address_data, {"address_id": "id"})

    # Process all other GeoJSONs in the directory
    for file_name in os.listdir(directory):
        if file_name.endswith(".geojson") and file_name not in ["unit.geojson", "anchor.geojson"]:
            file_path = os.path.join(directory, file_name)
            with open(file_path, 'r') as file:
                data = json.load(file)
            
            # Convert MultiPolygon to Polygon for 'level' and 'footprint'
            if file_name in ["level.geojson", "footprint.geojson", "venue.geojson"]:
                convert_to_polygon(data)
            # Convert MultiLineString to LineString for 'opening'. idk why QGIS makes it multi
            elif file_name == "opening.geojson":
                convert_to_linestring(data)
            
            # Generate UUIDs
            for feature in data["features"]:
                feature["id"] = str(uuid.uuid4())
            
            # Overwrite the original file with the transformed data
            with open(file_path, 'w') as output_file:
                json.dump(data, output_file, indent=4)
            print(f"Processed and overwrote {file_path}")

if __name__ == "__main__":
    directory = "C:\\Users\\amosk\\GitHub\\Geojson_2_IMDF_Transformer\\Transformed_QGIS_Geojsons"
    post_process_geojsons(directory)


Processed and overwrote C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\unit.geojson
Processed and overwrote C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\anchor.geojson
Processed and overwrote C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\address.geojson
Processed and overwrote C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\amenity.geojson
Processed and overwrote C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\building.geojson
Processed and overwrote C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\footprint.geojson
Processed and overwrote C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\level.geojson
Processed and overwrote C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transformed_QGIS_Geojsons\occupant.geojson
Processed and overwrote C:\Users\amosk\GitHub\Geojson_2_IMDF_Transformer\Transform